
# Image Processing and Feature Detection in Computer Vision

## Project Overview
This project explores fundamental image processing techniques for **edge detection** and **corner detection**—key concepts in computer vision. These techniques are widely used in applications such as object recognition, autonomous navigation, and medical imaging.

The project implements:
- **Edge Detection**: Using Gaussian smoothing and gradient computation to identify prominent edges.
- **Corner Detection**: Identifying key points in an image using second-order derivatives.

We use **convolution operations** with kernels to process images and extract meaningful features. The following sections present the implementation details and visualizations of the results.


In [ ]:
# Importing packages
import numpy as np
import matplotlib.pyplot as plt
import math

from scipy.signal import convolve2d
from skimage import io

import pickle

## Edge & Corner Detection

### Edge Detection

In this task, we implement an **edge detection algorithm** to identify prominent edges in an image. This is a crucial step in many computer vision applications, such as object detection and feature extraction.

### Steps:

1. **Smoothing:**  
   To reduce noise and prevent false edges, we apply a **9×9 Gaussian filter** with a standard deviation of **$\sigma = 1.2$** to smooth the image.

2. **Gradient Computation:**  
   After smoothing, we compute the image gradient in both the **horizontal ($G_x$)** and **vertical ($G_y$)** directions.  
   - The **gradient magnitude** is computed as:  
     $$
     |G| = \sqrt{G_x^2 + G_y^2}
     $$
   - The **gradient direction** is given by:  
     $$
     \theta = \tan^{-1} \left(\frac{G_y}{G_x}\right)
     $$

3. **Visualization:**  
   The results are visualized in four stages:
   - **Original Image**
   - **Smoothed Image**
   - **Gradient Magnitude**
   - **Gradient Direction**

**Dataset:** The input image used for this task is **`geisel.jpeg`**.

### Implementation Notes:
- The gradient direction can be calculated using `np.arctan2()`, which returns values in the range **[-180°, 180°]**.
- The gradient components (**$G_x$** and **$G_y$**) can be computed via convolution using `convolve2d()` from SciPy.
- The following Sobel-like kernels are used for computing the gradients:

  ![Convolution Kernels for Image Gradients](./figs/convolution_hint.png)


In [ ]:
def gaussian2d(filter_size=1, sig=1.0):
    """
    Creates 2D Gaussian kernel with side length `filter_size` and a sigma of `sig`.
    Source: https://stackoverflow.com/a/43346070
    """
    ax = np.arange(-filter_size // 2 + 1., filter_size // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sig))
    return kernel / np.sum(kernel)

def edge_detect(image):
    """
    Perform edge detection on the image.
    """
    smoothed = smooth(image)
    g_mag, g_theta = gradient(smoothed)
    return smoothed, g_mag, g_theta

def smooth(image):
    """
    Smoothes the image by using a 2D Gaussian kernel.
    
    Args:
        image: input image (h,w)

    Returns:
        smooth_image: smoothed version of input image (h,w)
    """
    smooth_image = np.zeros_like(image)
    filter_size = 9 # Kernel size
    sig = 1.2 # STD
    
    ### 
    gaussian_kernel = gaussian2d(filter_size, sig)
    # perform gaussian smoothing using convolution
    smooth_image = convolve2d(image, gaussian_kernel)

    ### END YOUR CODE
    return smooth_image

def gradient(image):
    """
    Computes a gradient direction image and a gradient magnitude image.

    Args:
        image: input image (h,w)

    Returns:
        g_mag: gradient magnitude (h,w)
        g_theta: gradient direction (h,w)
    """
    g_mag = np.zeros_like(image)
    g_theta = np.zeros_like(image)

    ### 
    k_x = 0.5 * np.array([[0,0,0], [1,0,-1],[0,0,0]])
    k_y = 0.5 * np.array([[0,1,0], [0,0,0], [0,-1,0]])

    df_dx = convolve2d(image, k_x)
    df_dy = convolve2d(image, k_y)

    g_mag = np.sqrt(df_dx**2 + df_dy**2)
    g_theta = np.arctan2(df_dy, df_dx)
    
    ### END YOUR CODE
    return g_mag, g_theta

# Plotting
# Load image in grayscale
image = io.imread("./imgs/geisel.jpeg", as_gray=True)
smoothed, g_mag, g_theta = edge_detect(image)
print('Original:')
plt.imshow(image, cmap='gray')
plt.show()

print('Smoothed:')
plt.imshow(smoothed, cmap='gray')
plt.show()

print('Gradient magnitude:')
plt.imshow(g_mag, cmap='gray')
plt.show()

print('Gradient direction:')
plt.imshow(g_theta, cmap='gray')
plt.show()

## Corner Detection

In this task, we implement a **corner detection algorithm** to identify corner-like features in an image. Corner detection is widely used in feature matching, object tracking, and 3D reconstruction.

### Approach:

To detect corners, we analyze the **second-moment matrix** of local image regions and extract the **minor eigenvalue** to identify high-intensity corner-like features.

### Steps:

1. **Compute Image Gradients**:  
   - Compute **horizontal ($G_x$)** and **vertical ($G_y$)** gradients using convolution.
   - Square and multiply gradients to obtain $G_x^2$, $G_y^2$, and $G_x G_y$.

2. **Compute Second-Moment Matrix**:  
   - Apply a **Gaussian filter (standard deviation $\sigma = 2.0$)** to smooth the gradient components.
   - Construct the second-moment matrix for each pixel.

3. **Eigenvalue Calculation**:  
   - Compute the **minor eigenvalue** of the second-moment matrix.
   - Identify the **top 100 corners** with the largest minor eigenvalues.

4. **Visualization**:  
   - Display the detected corners using the `show_corners_result` function.
   - Visualize the minor eigenvalue images using `show_eigen_images`.

**Dataset:** The input images used in this task are **`im0.png`** and **`im1.png`**, obtained from the [Middlebury Stereo Dataset](https://vision.middlebury.edu/stereo/data/).

### Implementation Notes:

- Image gradients are computed using **convolution** with Sobel-like kernels:

  ![Convolution Kernels for Image Gradients](./figs/convolution_hint.png)

- **Corner locations may not align perfectly** with visible corners in the image. This is expected because we detect regions with high minor eigenvalues rather than precise pixel-level corners.
- The algorithm may detect **"corner-like" features** in areas that mathematically satisfy the derivation, even if they are not visually obvious corners.
- When applying a sliding window for filtering, consider **center pixels** in odd-length windows for proper indexing.
- **Edge pixels** may contain noise and should be handled carefully when processing the image.



In [ ]:
def corner_detect(image, n_corners, smooth_std, window_size):
    """
    Detect corners on a given image

    Args:
        image: 2D grayscale image on which to detect corners
        n_corners: Total number of corners to be extracted
        smooth_std: Standard deviation of the Gaussian smoothing kernel
        window_size: Window size for Gaussian smoothing kernel,
                     corner detection, and nonmaximum suppresion

    Returns:
        minor_eig_image: The minor eigenvalue image (same shape as image)
        corners: Detected corners (in x-y coordinates) in a numpy array of shape (n_corners, 2)
    """
    corners = np.zeros((n_corners, 2))

    ### 
    # Step 1. Smooth image with Guassian filter to reduce noise
    gaussian_kernel = gaussian2d(window_size, smooth_std)
    # perform gaussian smoothing using convolution
    image = convolve2d(image, gaussian_kernel, mode="same")
    # Step 2. Compute magnitude of x and y gradients at each pixel
    k_x = 0.5 * np.array([[0,0,0], [1,0,-1],[0,0,0]])
    k_y = 0.5 * np.array([[0,1,0], [0,0,0], [0,-1,0]])

    i_x = convolve2d(image, k_x)
    i_y = convolve2d(image, k_y)
    
    # 3. Construct C in a window around each pixel
    # define border value for ignoring edge values of image
    minor_eig_image = np.zeros_like(image)

    border = window_size // 2
    for r in range(border, image.shape[0] - border):
        for c in range(border, image.shape[1] - border):
            # Create matrix C
            window_start_r = r - border
            window_end_r = r + border
            window_start_c = c - border
            window_end_c = c + border  # inclusive
    
            # Ensure window indices are within bounds
            window_start_r = max(0, window_start_r)
            window_end_r = min(image.shape[0] - 1, window_end_r)
            window_start_c = max(0, window_start_c)
            window_end_c = min(image.shape[1] - 1, window_end_c)
    
            # Extract windowed slice from i_x and i_y
            i_x_slice = i_x[window_start_r:window_end_r + 1, window_start_c:window_end_c + 1]
            i_y_slice = i_x[window_start_r:window_end_r + 1, window_start_c:window_end_c + 1]

            C = np.zeros((2,2))
            C[0][0] = np.sum(i_x_slice**2) # sum of Ix^2
            C[0][1] = np.sum(i_x_slice*i_y_slice) # sum of Ix*Iy
            C[1][0] = np.sum(i_x_slice*i_y_slice)
            C[1][1] = np.sum(i_y_slice**2) # sum of Iy^2

            # Compute eigenvalues
            eigenvalues = np.linalg.eigvals(C)
            
            # To calcualte min eigenvalue we take abs value
            minor_eigenvalue = np.min(np.abs(eigenvalues))

            minor_eig_image[r][c] = minor_eigenvalue

    suppressed_eig = np.zeros_like(minor_eig_image)
    # Apply nonmaximal suppression to minor_eig_image, move window by window size
    # Remove 
    for r in range(border + window_size, image.shape[0] - border, window_size):
        for c in range(border + window_size, image.shape[1] - border, window_size):
            # Create window
            window_start_r = r - border
            window_end_r = r + border
            window_start_c = c - border
            window_end_c = c + border  # inclusive
            
            # Ensure window indices are within bounds
            window_start_r = max(0, window_start_r)
            window_end_r = min(image.shape[0] - 1, window_end_r)
            window_start_c = max(0, window_start_c)
            window_end_c = min(image.shape[1] - 1, window_end_c)
            
            # Extract windowed slice and choose maximum
            slice = minor_eig_image[window_start_r:window_end_r + 1, window_start_c:window_end_c + 1]
            local_max = np.max(slice)
            max_index_window = np.unravel_index(np.argmax(slice), slice.shape)
            # Convert from index in window to index in array
            max_index_r, max_index_c = window_start_r + max_index_window[0], window_start_c + max_index_window[1]

            # Write to the suppressed matrix if the current candidate is the maximum eigenvalue within the grid cell
            suppressed_eig[max_index_r][max_index_c] = local_max
                
    # Find top n points in suppressed_eig
    top_corner_indexes = np.argsort(suppressed_eig, axis=None)
    sorted_indices = np.argsort(suppressed_eig, axis=None)[-n_corners:]
    # unflatten sorted indicies
    unraveled = np.unravel_index(sorted_indices, suppressed_eig.shape)
    # stack 2-tuple of y and x indicies
    corners = np.column_stack((unraveled[1], unraveled[0]))
    # print(corners.shape)

    # # get indicies of unflattened suprresed_eig
    # top_corner_indexes = np.unravel_index(top_corner_indexes, suppressed_eig.shape)
    # print(top_corner_indexes.shape)
    ### END YOUR CODE
    return minor_eig_image, corners

In [ ]:
def show_eigen_images(imgs):
    print("Minor Eigen value images")
    fig = plt.figure(figsize=(20, 20))
    # Plot image 1
    plt.subplot(1,2,1)
    plt.imshow(imgs[0], cmap='gray')
    plt.title('Image 0')

    # Plot image 2
    plt.subplot(1,2,2)
    plt.imshow(imgs[1], cmap='gray')
    plt.title('Image 1')
    
    plt.show()

def show_corners_result(imgs, corners, window_size):
    print("Detected Corners")
    fig = plt.figure(figsize=(20, 20))
    ax1 = fig.add_subplot(221)
    ax1.imshow(imgs[0], cmap='gray')
    ax1.scatter(corners[0][:, 0], corners[0][:, 1], s=25, edgecolors='r', facecolors='none')
    ax1.title.set_text("Image 0")

    ax2 = fig.add_subplot(222)
    ax2.imshow(imgs[1], cmap='gray')
    ax2.scatter(corners[1][:, 0], corners[1][:, 1], s=25, edgecolors='r', facecolors='none')
    ax2.title.set_text("Image 1")
    
    plt.show()

### You may want to modify these parameters here..

In [ ]:
# Detect corners on the two provided images

# You may want to change these parameters accordingly
n_corners = 100
smooth_std = 2.0
window_size = 13

# Read images and detect corners on images
imgs = []
eig_imgs = []
corners = []
for i in range(2):
    img = io.imread(f"./imgs/im{i}.png", as_gray=True)
    imgs.append(img)
    #img = io.imread('im' + str(i) + '.png')
    minor_eig_image, corners_vals = corner_detect(imgs[-1], n_corners, smooth_std, window_size)
    eig_imgs.append(minor_eig_image)
    corners.append(corners_vals)

# Show the results
# This may take a few seconds to run
show_eigen_images(eig_imgs)
show_corners_result(imgs, corners, window_size)

## Task 2: Theory and Solving Example Problems
Consider two cameras whose image planes are the z=2 plane, and whose focal  are at (-2, 0, 0) and (8, 0, 0). See Fig 1.1 below. We'll call a point in the first camera (x, y), and a point in the second camera (u, v). Points in each camera are relative to the camera center. So, for example if (x, y) = (0, 0), this is really the point (-2, 0, 2) in world coordinates, while if (u, v) = (0, 0) this is the point (8, 0, 2).

Suppose the point $(x,y)=(3,3)$ is matched to the point $(u,v)=(1,3)$. What is the 3D location of this point?



From the positions of the focal : $C_1 = (-2, 0, 0), C_2 = (8, 0, 0)$. 

Next we convert the coordinates (x,y) into world cordinates $x_1^w  = (3+ (-2) ,3+0,2) = (1,3,2)$ and the coordinates (u,v) is  $x_2^w  = (1+8,3+0,2) = (9,3,2)$


The direction vector which  in the direction of backprojecting a raw from the image plane to the focal point is:
- $x_1^w - C_1 = (1-(-2),3-0,2-0) = (3,3,2)$
- $x_2^w - C_2 = (9-(8),3-0,2-0) = (1,3,2)$

Therefore the backprojected rays can be defined as:
- $R_1(t) = C_1 + t * (3,3,2)$
- $R_2(s) = C_2 + s * (1,3,2)$

To find the point of intersection:
- $R_1(t) = R_2(s)$ 
- $(-2, 0, 0) + (3t, 3t, 2t)  = (8, 0, 0) + (1s,3s,2s)$

Using system of equations:
- $-2+3t = 8 + 1s$
- $3t =  3s$
- $2t = 2s$Ω

Thus $t = s$:
- $-2 + 3t = 8 + 1t$
- $2t = 10$
- $t = 5$

Substituting t into $R_1$ we get the 3d point:
- $(-2, 0, 0) + 5 * (3,3,2) = (-2 + 15, 0 + 15, 0 + 10) = (13, 15, 10)$

### The Epipolar Constraint

Suppose two cameras fixate on a point $P$ in space such that their principal axes intersect at that point. (See the fig. 1.2 below.) Show that if the image coordinates are normalized so that the coordinate origin (0, 0) coincides with the principal point, then the $F_{33}$ element of the fundamental matrix is zero.

![fig 2.2](./figs/ec_diagram.png)

In the figure, $C1$ and $C2$ are the optical centers. The principal axes intersect at point $P$.



By definition: $x'^T F x = 0$
Next we convert the  on the camera planes into homogenous coordinates:
- $x = (u,v,1)^T = (0,0,1)$
- $x' = (u',v',1)^T = (0,0,1)$

Expanding the $x'^T F x = 0$ equation into a linear equation we get:


- $uu' f_{11} + vu' f_{12} + u' f_{13} + uv' f_{21} + vv' f_{22} + v' f_{23} + uf_{31} + vf_{32} + f_{33}  = 0$

- $0 + 0 + 0 + 0 + 0 + 0 + 0 + 0 + f_{33} = 0$
- Therefore $ f_{33} = 0$

### Essential Matrix

Suppose a stereo rig is formed of two cameras: the rotation matrix and translation vector are given to you. Please write down the essential matrix. Also, compute the rank of the essential matrix using SVD, i.e., the number of nonzero singular values. (Note that if you get a singular value $s$ of a very small number in your calculation, e.g., $s<=1e-15$, you can treat it as zero singular value). 


$$ R=
\begin{bmatrix}
\frac{\sqrt{3}}{2} & \frac{1}{2} & 0 \\
-\frac{1}{2} & \frac{\sqrt{3}}{2} & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

$$ t=
\begin{bmatrix}
4 \\ 2 \\3
\end{bmatrix}
$$



First we turn translation vector into a 3x3 matrix so that we can take the cross product of $t$ and $p$ through matrix multiplication: $ [t]_x = 
\begin{pmatrix}
0 & -3 & 2 \\
3 & 0 & -4 \\
-2 & 4 & 0 \\
\end{pmatrix}$

Next we multiply $[t]_X$ with $R$:
$\begin{bmatrix}
0 & -3 & 2 \\
3 & 0 & -4 \\
-2 & 4 & 0 \\
\end{bmatrix} * \begin{bmatrix}
\frac{\sqrt{3}}{2} & \frac{1}{2} & 0 \\
-\frac{1}{2} & \frac{\sqrt{3}}{2} & 0 \\
0 & 0 & 1
\end{bmatrix} =\begin{bmatrix}
\frac{3}{2} & \frac{-3\sqrt{3}}{2} & 2 \\
-\frac{3\sqrt{3}}{2} & \frac{3}{2} & -4 \\
-\sqrt{3} - 2 & 2\sqrt{3}-1 & 0
\end{bmatrix} $

In [ ]:
# Finding rank using SVD
t_x = np.array([[0, -3, 2],
              [3, 0, -4],
              [-2, 4, 0]])

R = np.array([[np.sqrt(3)/2, 1/2, 0],
              [-1/2, np.sqrt(3)/2, 0],
              [0, 0, 1]])

# Perform matrix multiplication
E = np.dot(t_x, R)

# Perform SVD
U, S, V = np.linalg.svd(E)

# Count number of singular values in S
rank = np.sum(S > 0.0000001)



print(rank)

Therefore rank is 2

## Window-Based Similarity Matching: SSD & NCC

In this task, we implement **window-based similarity matching** techniques to compare image patches. These methods are commonly used in **stereo vision, template matching, and optical flow estimation**.

### Methods:

1. **SSD (Sum Squared Distance) Matching**:  
   The Sum Squared Distance (SSD) metric computes the difference between two image patches by summing the squared differences of corresponding pixel values. A lower SSD value indicates a better match.

   **Formula:**
   $$
   SSD = \sum_{x,y} |W_1(x,y) - W_2(x,y)|^2
   $$

2. **NCC (Normalized Cross-Correlation) Matching**:  
   The Normalized Cross-Correlation (NCC) metric measures the similarity between two windows by normalizing their pixel intensities and computing a correlation score. A higher NCC value (closer to **1**) indicates a stronger match.

   **Formula:**
   $$
   NCC = \frac{\sum_{x,y} (W_1(x,y) - \bar{W}_1)(W_2(x,y) - \bar{W}_2)}
   {\sqrt{\sum_{x,y} (W_1(x,y) - \bar{W}_1)^2} \cdot \sqrt{\sum_{x,y} (W_2(x,y) - \bar{W}_2)^2}}
   $$

### Implementation:

- **`ssd_match` Function**:  
  Computes the **Sum Squared Distance (SSD)** for two given image windows.

- **`ncc_match` Function**:  
  Computes the **Normalized Cross-Correlation (NCC)** for two given image windows.

### Applications:
- **Stereo Matching**: Identifying corresponding points in left-right image pairs.
- **Template Matching**: Locating an object in an image based on similarity.
- **Feature Tracking**: Matching keypoints across frames in video sequences.

### Implementation Notes:
- SSD is **sensitive to intensity differences** and does not handle illumination changes well.
- NCC is **scale-invariant** and better suited for cases where intensity variations exist.



In [ ]:
def ssd_match(img1, img2, c1, c2, R):
    """
    Compute SSD of two windows.
    
    Args:
        img1: Image 1.
        img2: Image 2.
        c1: Center (in x-y coordinates) of the window in image 1.
        c2: Center (in x-y coordinates) of the window in image 2.
        R: R is the radius of the patch, 2 * R + 1 is the window size

    Returns:
        SSD matching score for two input windows (a scalar value).
    """
    ### 
    # 1. Find sum of window in image 1
    window_start_r = c1[1] - R
    window_end_r = c1[1] + R
    window_start_c = c1[0] - R
    window_end_c = c1[0] + R

    window_1 = img1[window_start_r:window_end_r + 1, window_start_c:window_end_c + 1]

    # # 2. Find sum of window in image 2
    window_start_r = c2[1] - R
    window_end_r = c2[1] + R
    window_start_c = c2[0] - R
    window_end_c = c2[0] + R
    window_2 = img2[window_start_r:window_end_r + 1, window_start_c:window_end_c + 1]

    # # 3. Calculate and return matching score
    matching_score = np.sum(abs(window_1 - window_2) ** 2)

    ### END YOUR CODE
    return matching_score

In [ ]:
# Here is the code for you to test your implementation
img1 = np.array([[1, 2, 3, 4], [4, 5, 6, 8], [7, 8, 9, 4]])
img2 = np.array([[1, 2, 1, 3], [6, 5, 4, 4], [9, 8, 7, 3]])

result1 = ssd_match(img1, img2, np.array([1, 1]), np.array([1, 1]), 1)
# should print 20
assert(result1 == 20)
print(f"Result 1: {result1}")

result2 = ssd_match(img1, img2, np.array([2, 1]), np.array([2, 1]), 1)
# should print 30
assert(result2 == 30)
print(f"Result 2: {result2}")

result3 = ssd_match(img1, img2, np.array([1, 1]), np.array([2, 1]), 1)
# should print 46
assert(result3 == 46)
print(f"Result 3: {result3}")

## Window-Based Similarity Matching: SSD & NCC

In this task, we implement **window-based similarity matching** techniques to compare image patches. These methods are commonly used in **stereo vision, template matching, and optical flow estimation**.

### Methods:

1. **SSD (Sum Squared Distance) Matching**:  
   The Sum Squared Distance (SSD) metric computes the difference between two image patches by summing the squared differences of corresponding pixel values. A lower SSD value indicates a better match.

   **Formula:**
   $$
   SSD = \sum_{x,y} |W_1(x,y) - W_2(x,y)|^2
   $$

2. **NCC (Normalized Cross-Correlation) Matching**:  
   The Normalized Cross-Correlation (NCC) metric measures the similarity between two windows by normalizing their pixel intensities and computing a correlation score. A higher NCC value (closer to **1**) indicates a stronger match.

   **Formula:**
   $$
   NCC = \frac{\sum_{x,y} (W_1(x,y) - \bar{W}_1)(W_2(x,y) - \bar{W}_2)}
   {\sqrt{\sum_{x,y} (W_1(x,y) - \bar{W}_1)^2} \cdot \sqrt{\sum_{x,y} (W_2(x,y) - \bar{W}_2)^2}}
   $$

### Implementation:

- **`ssd_match` Function**:  
  Computes the **Sum Squared Distance (SSD)** for two given image windows.

- **`ncc_match` Function**:  
  Computes the **Normalized Cross-Correlation (NCC)** for two given image windows.

### Applications:
- **Stereo Matching**: Identifying corresponding points in left-right image pairs.
- **Template Matching**: Locating an object in an image based on similarity.
- **Feature Tracking**: Matching keypoints across frames in video sequences.

### Implementation Notes:
- SSD is **sensitive to intensity differences** and does not handle illumination changes well.
- NCC is **scale-invariant** and better suited for cases where intensity variations exist.



In [ ]:
def ncc_match(img1, img2, c1, c2, R):
    """
    Compute NCC given two windows.

    Args:
        img1: Image 1.
        img2: Image 2.
        c1: Center (in image coordinate) of the window in image 1.
        c2: Center (in image coordinate) of the window in image 2.
        R: R is the radius of the patch, 2 * R + 1 is the window size

    Returns:
        NCC matching score for two input windows.

    """
    ### 
    # Calculate W~1
    window_start_r = c1[1] - R
    window_end_r = c1[1] + R
    window_start_c = c1[0] - R
    window_end_c = c1[0] + R

    window_1 = img1[window_start_r:window_end_r + 1, window_start_c:window_end_c + 1]
    W_bar = np.mean(window_1)
    numerator = window_1 - W_bar
    denominator = (window_1 - W_bar) ** 2
    denominator = np.sum(denominator)
    denominator = np.sqrt(denominator)
    W_1 = numerator / denominator

    # Calculate W~2
    window_start_r = c2[1] - R
    window_end_r = c2[1] + R
    window_start_c = c2[0] - R
    window_end_c = c2[0] + R

    window_2 = img2[window_start_r:window_end_r + 1, window_start_c:window_end_c + 1]
    W_bar = np.mean(window_2)
    numerator = window_2 - W_bar
    denominator = (window_2 - W_bar) ** 2
    denominator = np.sum(denominator)
    denominator = np.sqrt(denominator)
    W_2 = numerator / denominator
    

    ## Multiply W~1 and W~2 and sum resulting matrix
    matching_score = np.sum(W_1 * W_2)

    ### END YOUR CODE

    return matching_score

In [ ]:
# Here is the code for you to test your implementation
img1 = np.array([[1, 2, 3, 4], [4, 5, 6, 8], [7, 8, 9, 4]])
img2 = np.array([[1, 2, 1, 3], [6, 5, 4, 4], [9, 8, 7, 3]])

result4 = ncc_match(img1, img2, np.array([1, 1]), np.array([1, 1]), 1)
# should print 0.8546
print(f"Result 4: {result4}")

result5 = ncc_match(img1, img2, np.array([2, 1]), np.array([2, 1]), 1)
# should print 0.8457
print(f"Result 5: {result5}")

result6 = ncc_match(img1, img2, np.array([1, 1]), np.array([2, 1]), 1)
# should print 0.6258
print(f"Result 6: {result6}")

## Feature Correspondence: Naive Matching

With the detected corners and the **Normalized Cross-Correlation (NCC) matching function**, we now aim to establish correspondences between features in two images. 

### Approach:

A **naive matching strategy** is used to identify the best correspondence between two sets of detected corners:
1. **For each corner in Image 1**, compute the **NCC match score** with every detected corner in Image 2.
2. **Select the best match** based on the highest NCC score.
3. **Apply a threshold**: If the highest NCC score is below a certain value, no match is assigned.

### Implementation:

- **`naive_matching` Function**:  
  - Iterates through detected corners in Image 1.
  - Finds the **best match** in Image 2 using the **NCC metric**.
  - Returns a set of **matched pairs**.

- The function is called as follows:
  ```python
  matches = naive_matching(corners_img1, corners_img2, image1, image2, threshold)


In [ ]:
def naive_matching(img1, img2, corners1, corners2, R, NCCth):
    """
    Compute NCC given two windows.

    Args:
        img1: Image 1.
        img2: Image 2.
        corners1: Corners in image 1 (nx2)
        corners2: Corners in image 2 (nx2)
        R: NCC matching radius
        NCCth: NCC matching score threshold

    Returns:
        matching: NCC matching returns a list of tuple (c1, c2), 
                  c1 is the 1x2 corner location in image 1, 
                  c2 is the 1x2 corner location in image 2. 

    """

    matching = []
    
    ### 
    # 1. For each corner1 in img1, we compute the NCC between it and every corner2 in img2 and save the best match
    for c1 in corners1:
        # define max and pair for c1
        max_match = ncc_match(img1, img2, c1, corners2[0], R)
        correspondence_pair = (c1, corners2[0])
        for c2 in corners2:
            current_ncc = ncc_match(img1, img2, c1, c2, R)
            if current_ncc > max_match:
                max_match = current_ncc
                correspondence_pair =  (c1, c2)
        # add corrispondence pair between highest c1 and c2 to matching by stacking if max_match exeeds the threshold
        if max_match >= NCCth:
            matching.append(correspondence_pair)
            # print(correspondence_pair)
            
    # print(matching)

    return matching

In [ ]:
def rgb2gray(rgb):
    """ 
    Convert rgb image to grayscale.
    """
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

# Detect corners on warrior and matrix sets
# You are free to modify the parameters if you wish
n_corners = 20
smooth_std = 1
window_size = 19

# Read images and detect corners on the images
imgs_mat = []
crns_mat = []
imgs_war = []
crns_war = []

for i in range(2):
    img_mat = io.imread('./imgs/p4/matrix/matrix' + str(i) + '.png')
    imgs_mat.append(rgb2gray(img_mat))
    img_war = io.imread('./imgs/p4/warrior/warrior' + str(i) + '.png')
    imgs_war.append(rgb2gray(img_war))

In [ ]:
# Match corners (May need to modify the threshold below)
crnsmatf=open('./imgs/p3/crns_mat.pkl','rb')
crns_mat=pickle.load(crnsmatf)
crnswarf=open('./imgs/p3/crns_war.pkl','rb')
crns_war=pickle.load(crnswarf)
R = 120
NCCth = 0.5  # MAY NEED TO MODIFY YOUR THRESHOLD HERE
matching_mat = naive_matching(imgs_mat[0]/255, imgs_mat[1]/255, crns_mat[0], crns_mat[1], R, NCCth)
matching_war = naive_matching(imgs_war[0]/255, imgs_war[1]/255, crns_war[0], crns_war[1], R, NCCth)

In [ ]:
# Plot matching result
def show_matching_result(img1, img2, matching):
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(np.hstack((img1, img2)), cmap='gray') # two dino images are of different sizes, resize one before use
    for p1, p2 in matching:
        plt.scatter(p1[0], p1[1], s=35, edgecolors='r', facecolors='none')
        plt.scatter(p2[0] + img1.shape[1], p2[1], s=35, edgecolors='r', facecolors='none')
        plt.plot([p1[0], p2[0] + img1.shape[1]], [p1[1], p2[1]])
    #plt.savefig('matching.png')
    plt.show()

print("Number of Corners:", n_corners)
show_matching_result(imgs_mat[0], imgs_mat[1], matching_mat)
show_matching_result(imgs_war[0], imgs_war[1], matching_war)

## Epipolar Geometry & Fundamental Matrix Estimation

This section explores **epipolar geometry**, a fundamental concept in **stereo vision** that defines geometric constraints between two images taken from different viewpoints. Instead of relying on brute-force matching, **epipolar constraints** allow us to efficiently find corresponding points along predefined lines, significantly reducing computational complexity.

### Understanding Epipolar Geometry:
- **Epipolar Constraint**: Corresponding points between two images must lie on **epipolar lines**.
- **Fundamental Matrix ($F$)**: A **$3 \times 3$ matrix** that relates corresponding points between two views.

By estimating **$F$**, we can:
- **Visualize epipolar lines** for better understanding of correspondences.
- **Improve feature matching algorithms** by reducing search space.
- **Rectify images** for stereo vision applications (beyond this implementation).

---

### Computing the Fundamental Matrix (8-Point Algorithm)

We use the **8-point algorithm** to estimate the fundamental matrix **$F$** from corresponding points.

1. **Construct the constraint matrix ($A$)**:  
   - Given **8 or more corresponding points**, we formulate a system of linear equations **$Af = 0$**.
  
2. **Solve using Singular Value Decomposition (SVD)**:  
   - The solution **$f$** corresponds to the **smallest singular vector** of **$A$**, obtained via **SVD**.
   - Note: Numpy’s `np.linalg.svd()` returns **$V^T$**, so we use the last row of **$V^T$**.

3. **Enforce rank-2 constraint**:  
   - The fundamental matrix **$F$** should have rank 2 (handled in the provided code).

---

### Implementation:

- **`compute_fundamental` Function**:
  - Implements the **8-point algorithm** to estimate **$F$**.
  - Uses **SVD** to solve **$Af = 0$**.

- **Visualization**:
  - **Epipolar lines** are drawn on both images.
  - The results help us understand **how points correspond between two views**.

---

### Practical Applications:
- **Stereo Matching**: Used to **rectify images** for depth estimation.
- **3D Reconstruction**: Helps recover **scene structure** from multiple views.
- **Camera Calibration**: Essential in **multi-camera systems**.

### Implementation Notes:
- **Ensure that point correspondences are normalized** before computing **$F$**.
- **Epipolar lines should be plotted** to validate the correctness of the fundamental matrix.
- **Improved methods (beyond 8-point)** include the **7-point and RANSAC-based approaches** for handling noise.




In [ ]:
F = np.array([[1, 6, 9],
              [9, 6, 6],
              [8, 4, 1]])

# constrain F
# make rank 2 by zeroing out last singular value
U,S,V = np.linalg.svd(F)
S[2] = 0
F_prime = np.dot(U,np.dot(np.diag(S),V))

In [ ]:
from scipy.linalg import null_space

# Find g so F'g = 0 by using null space
g = null_space(F_prime)
# Invert F_prime for h
h = null_space(F_prime.T)

print("g",g)
print("h",h)

In [ ]:
def compute_fundamental(x1, x2):
    """
    Computes the fundamental matrix from corresponding points 
    (x1,x2 3xn arrays) using the 8 point algorithm.
        
    Construct the A matrix according to lecture
    and solve the system of equations for the entries of the fundamental matrix.

    Args:
        x1: Point correspondences from img1 (3xn)
        x2: Point correspondences from img2 (3xn)
    
    Returns:
        Fundamental Matrix (3x3)
    """
    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don't match.")
    
    ### 
    # Build cols of A
    u = x1[0, :] # col 7
    u_prime = x2[0, :] # col 3
    v = x1[1, :].T # col 8
    v_prime = x2[1, :] # col 6

    uu_prime = u * u_prime # col 1,
    vu_prime = v * u_prime # col 2
    uv_prime = u * v_prime # col 4
    vv_prime = v * v_prime # col 5
    ones = np.ones_like(vv_prime) # col 9
    # print(v.shape)
    
    # Build A using the corrispondences (n x 9), concatenate rows top to bottom then transpose
    A = np.vstack((uu_prime, vu_prime, u_prime, uv_prime, vv_prime, v_prime, u, v, ones))
    A = A.T

    # Solve system of equations using svd
    U,S,Vt = np.linalg.svd(A)

    # Extract fundemental matrix from last column of V
    F = np.reshape(Vt.T[:, -1], (3,3))

    ### END YOUR CODE
    
    # constrain F
    # make rank 2 by zeroing out last singular value
    U,S,V = np.linalg.svd(F)
    S[2] = 0
    F = np.dot(U,np.dot(np.diag(S),V))
    
    return F/F[2,2]

In [ ]:
def fundamental_matrix(x1,x2):
    # Normalization of the corner points is handled here
    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don't match.")

    # normalize image coordinates
    x1 = x1 / x1[2]
    mean_1 = np.mean(x1[:2],axis=1)
    S1 = np.sqrt(2) / np.std(x1[:2])
    T1 = np.array([[S1,0,-S1*mean_1[0]],[0,S1,-S1*mean_1[1]],[0,0,1]])
    x1 = np.dot(T1,x1)
    
    x2 = x2 / x2[2]
    mean_2 = np.mean(x2[:2],axis=1)
    S2 = np.sqrt(2) / np.std(x2[:2])
    T2 = np.array([[S2,0,-S2*mean_2[0]],[0,S2,-S2*mean_2[1]],[0,0,1]])
    x2 = np.dot(T2,x2)

    # compute F with the normalized coordinates
    F = compute_fundamental(x1,x2)

    # reverse normalization
    F = np.dot(T2.T,np.dot(F,T1))

    return F/F[2,2]

In [ ]:
# Here is the code for you to test your implementation
cor1 = np.load("./imgs/p4/"+'dino'+"/cor1.npy")
cor2 = np.load("./imgs/p4/"+'dino'+"/cor2.npy")
print(fundamental_matrix(cor1,cor2))
# Should print 
#[[ 4.00502510e-07 -2.69900666e-06  1.37819769e-03]
# [ 3.09619039e-06 -1.00972419e-08 -7.29675791e-03]
# [-2.86966053e-03  6.70452915e-03  1.00000000e+00]]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


tuvalu = np.array([[1, 9, 0, 1]])
vectors = np.array([
    [0, 9, 0, 0],
    [0, 1, 0, 0],
    [1, 50, 0, 0],
    [1, 6, 0, 1],
    [1, 1, 1, 0]
])

cosine_similarities = cosine_similarity(tuvalu, vectors).flatten()

cosine_similarities

## Visualizing Epipolar Geometry

Once the **fundamental matrix ($F$)** is computed, we can **visualize epipolar geometry** by plotting **epipolar lines** on both images of a stereo pair. This helps confirm the correctness of our **$F$** estimation.

### Purpose:
- **Validate the computed fundamental matrix** by checking if corresponding points lie on their respective epipolar lines.
- **Gain insights into stereo geometry** by observing how points are constrained along epipolar lines.
- **Ensure accurate correspondences** before using epipolar constraints for further tasks like rectification.

### Visualization:

- We use the function **`plot_epipolar_lines`** to draw the epipolar lines on both images.
- Run the provided cells and analyze the results.

### Expected Output:
- Epipolar lines should **pass through corresponding points** in both images.
- The visualized **epipolar lines for "matrix" and "warrior"** should resemble the example figures below.

  ![Dino Epipolar 1](./figs/dinoEpi1.png)
  ![Dino Epipolar 2](./figs/dinoEpi2.png)

### Notes:
- **If the epipolar lines do not align correctly**, revisit the **fundamental matrix computation** in `compute_fundamental`.
- **Outliers in feature matching** can cause misalignment—this is common in real-world datasets.



In [ ]:
def plot_epipolar_lines(img1, img2, cor1, cor2):
    """
    Plot epipolar lines on image given image, corners

    Args:
        img1: Image 1.
        img2: Image 2.
        cor1: Corners in homogeneous image coordinate in image 1 (3xn)
        cor2: Corners in homogeneous image coordinate in image 2 (3xn)

    """
    
    assert cor1.shape[0] == 3
    assert cor2.shape[0] == 3
    assert cor1.shape == cor2.shape
    
    F = fundamental_matrix(cor1, cor2)
        
    # epipole in image 1 is the solution to Fe = 0
    U,S,V = np.linalg.svd(F)
    e1 = V[-1]
    e1 /= e1[-1]
    
    # epipole in image 2 is the solution to F.Te = 0
    U,S,V = np.linalg.svd(F.T)
    e2 = V[-1]
    e2 /= e2[-1]

    plot_epipoles = False
    
    # Plot epipolar lines in the first image
    # There is an epipolar line for each corner
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(img1, cmap='gray')
    h, w = img1.shape[:2]
    for c2 in cor2.T:
        # epipolar line is (F.T * c2) dot (x, y, 1) = 0
        epi_line = np.dot(F.T, c2)
        a, b, c = epi_line # ax + by + c = 0, y = -a/b * x - c/b
        x = np.arange(w)
        y = (-a / b) * x - (c / b)
        x = np.array([x[i] for i in range(x.size) if y[i] >=0 and y[i] < h - 1])
        y = np.array([y[i] for i in range(y.size) if y[i] >=0 and y[i] < h - 1])
        plt.plot(x, y, 'b', zorder=1)
        
    plt.scatter(cor1[0], cor1[1], s=50, edgecolors='b', facecolors='r', zorder=2)
    
    if plot_epipoles:
        plt.scatter([e1[0]], [e1[1]], s=75, edgecolors='g', facecolors='y', zorder=3)
    plt.show()
    
    # Plot epipolar lines in the second image
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(img2, cmap='gray')
    h, w = img2.shape[:2]
    
    for c1 in cor1.T:
        # epipolar line is (F * c1) dot (x, y, 1) = 0
        epi_line = np.dot(F, c1)
        a, b, c = epi_line
        x = np.arange(w)
        y = (-a / b) * x - (c / b)
        x = np.array([x[i] for i in range(x.size) if y[i] >=0 and y[i] < h - 1])
        y = np.array([y[i] for i in range(y.size) if y[i] >=0 and y[i] < h - 1])
        plt.plot(x, y, 'b', zorder=1)
    
    plt.scatter(cor2[0], cor2[1], s=50, edgecolors='b', facecolors='r', zorder=2)
    
    if plot_epipoles:
        plt.scatter([e2[0]], [e2[1]], s=75, edgecolors='g', facecolors='y', zorder=3)
    plt.show()

In [ ]:
# Replace images and corners with those of matrix and warrior
imgids = ["dino", "matrix", "warrior"]
for imgid in imgids:
    I1 = io.imread("./imgs/p4/"+imgid+"/"+imgid+"0.png")
    I2 = io.imread("./imgs/p4/"+imgid+"/"+imgid+"1.png")
    cor1 = np.load("./imgs/p4/"+imgid+"/cor1.npy")
    cor2 = np.load("./imgs/p4/"+imgid+"/cor2.npy")
    plot_epipolar_lines(I1,I2,cor1,cor2)

## Conclusions and Insights

This project successfully explored various computational methods and algorithmic solutions. The key takeaways include:

- The importance of optimizing algorithms for efficiency and scalability.
- How different data structures impact computational performance.
- The significance of visualization in interpreting computational results.

### Future Enhancements:
- Applying these techniques to real-world datasets for validation.
- Exploring parallel computing techniques for further optimization.
- Expanding the study to include machine learning models and predictive analytics.

This project serves as a foundation for further exploration in computational sciences and software engineering.


## Key Takeaways

Through this project, we successfully implemented:
- **Edge detection** using Gaussian smoothing and gradient computation.
- **Feature detection** through convolution and filtering techniques.
- **Image processing techniques** essential in computer vision.

### Future Enhancements:
- Implement Canny edge detection for better results.
- Extend feature detection to work with real-world images.
- Experiment with different kernel sizes and parameters.

This project provides a solid foundation for deeper exploration into **computer vision** and **machine learning**.
